In [ ]:
# Auto-Reload Setup für Entwicklung
# Automatisches Neuladen aller Module bei Änderungen
%load_ext autoreload
%autoreload 2

# Persönliche Dokumentation für das Tool
## Anforderungen
- CSV Exportdatei aus mLife entgegen nehmen ("gesamte_akte.csv")
- Alle (oder zumindest für RedCap erforderliche) Daten in ein Dataframe parsen
- Darstellung und Konfiguration des Exports mit Streamlit
- Export als CSV, welche in RedCap importiert werden kann

## Übersicht Datenfluss
```
CSV-Import (gesamte_akte.csv)
    ↓
DataParser (services/data_parser.py)
    ↓
StateProvider (DataManager ↔ QueryManager)
    ↓
Value Aggregation (services/value_aggregation/…)
    ↓
Views & Exporte (views/…, exports)
```

## Import & State-Aufbau
- aus mLife wird eine CSV-Datei der gesamten_akte exportiert; der Delimiter kann zwischen ";" und "|" gewählt werden und wird automatisch erkannt
### StateProvider
- `state_provider.state_provider StateProvider` initialisiert sich beim Import und exportiert die Singleton-Instanz `state_provider`
    - `DataManager` übernimmt Parsing, Mutationen und alle Schreiboperationen
    - `QueryManager` kapselt lesende Operationen, Filter, Aggregationen und Zeitbereichsabfragen
    - `schemas.app_state_schemas.app_state AppState` hält `ParsedData`, `Views`, `UiState`, Metadaten und Zeitbereiche
- `state_provider.parse_data_to_state()` delegiert an den `DataManager`
    - CSV + Delimiter werden an `services.data_parser DataParser` übergeben (`DataParser(file, delimiter)`)
    - `DataManager` ruft die Parser-Methoden auf, erzeugt DataFrames für jede Kategorie (Labor, Vitalwerte, NIRS, …)
    - Die resultierenden DataFrames werden zu einer `ParsedData`-Instanz zusammengeführt und in `AppState.parsed_data` gespeichert
    - `DataManager` leitet initiale Zeitbereiche, Record-ID, nearest-times, etc. an das AppState-Objekt weiter
### Verarbeitung im `DataParser`
- Beim Initialisieren wird das File nur gelesen; auf den ersten Parser-Aufruf hin folgen `_clean_csv()` und `_split_blocks()`
- `DataParser` kombiniert `DataParserBase` und spezialisierte Mixins (Medication, Devices, Fluidbalance, …) in einer Datei
- Wichtige Parser-Methoden erzeugen `pandas.DataFrame`-Objekte, validiert über `schemas.parse_schemas`
    - `_parse_table_data()` (Vitals, Labor, Respiratory)
    - `parse_nirs_logic()`
    - `parse_medication_logic()`
    - `parse_fluidbalance_logic()`
    - `parse_from_all_patient_data()` / `parse_all_patient_data()`
    - `parse_respiratory_data()`
### Datenimport im UI
#### Homepage
- Upload der `gesamte_akte.csv` und Auswahl des Delimiters
- Wird nur angezeigt, wenn `AppState.parsed_data` **nicht** gesetzt ist
#### Sidebar
- Zugriff auf Views, Record-ID, ausgewählten Zeitraum, Value-Strategie
#### Overview
- Gesamtzahl der parsed Datensätze
- Zeitraum der Aufzeichnungen via `state_provider.query_manager.get_time_range()`
- Gerätespezifische Zeiträume über `state_provider.get_device_time_ranges()`

## Aggregation & Queries
- `QueryManager` liefert alle lesenden Operationen und Filter auf den `ParsedData`-DataFrames
- `state_provider.query_data()` ist ein Kurzschluss auf `QueryManager.query_data()` und akzeptiert Filter (timestamp, parameter, category, value_strategy, …)
- Aggregatoren (z.B. `services/value_aggregation/lab_aggregator.py`) nutzen ausschließlich diese Query-Schnittstelle
- Datumskonvertierungen und Bounds werden durch `services.utils` vereinheitlicht (`coerce_to_datetime`, `expand_date_range_to_bounds`, …)

In [ ]:
# Alle öffentlichen Methoden von state_provider, QueryManager und DataManager inspizieren
from state_provider.state_provider import state_provider
def show_available_methods(obj):
    return [name for name in dir(obj) if not name.startswith("_")]
print("StateProvider:")
for method in show_available_methods(state_provider):
    print(f"  - {method}")
print("\nQueryManager:")
for method in show_available_methods(state_provider.query_manager):
    print(f"  - {method}")
print("\nDataManager:")
for method in show_available_methods(state_provider.data_manager):
    print(f"  - {method}")

Verfügbare StateProvider Methoden:
   1. data_parser
   2. get_device_time_ranges
   3. get_respiration_type
   4. get_respiratory_value
   5. get_selected_view
   6. get_state
   7. get_time_of_mcs
   8. get_time_range
   9. get_vasoactive_agents_df
  10. get_vitals_value
  11. has_device_past_24h
  12. has_mcs_records_past_24h
  13. has_parsed_data
  14. parse_data_to_state
  15. query_data
  16. reset_state
  17. save_state
  18. set_selected_time_range
  19. update_state


Die zentrale Schnittstelle bleibt `StateProvider.query_data()` (alias für `QueryManager.query_data()`).
- Neben ganzen Kategorien erlauben Filter differenzierte Query-Strings
- unterstützte `value_strategy`: "first", "last", "median", "mean", "nearest"
- bei `"nearest"` zusätzlich `nearest_time: datetime.time` erforderlich
- der Rückgabewert ist immer ein `pandas.DataFrame` (leer, falls kein State vorliegt)

In [ ]:
from datetime import datetime, time
from services.delimiter_auto_detecion import detect_delimiter
from state_provider.state_provider import state_provider

file = "data/gesamte_akte2.csv"
delimiter = detect_delimiter(file)
state = state_provider.parse_data_to_state(file, delimiter)  # delegiert an DataManager

# QueryManager liefert DataFrames je Kategorie
vitals = state_provider.query_data("vitals")
ecmo = state_provider.query_data("ecmo")

test_date = datetime(2025, 9, 15).date()
art = state_provider.query_data("vitals", {"timestamp": test_date, "parameter": "ARTs", "value_strategy": "median"})

heparin = state_provider.query_data("medication", {"medication": "heparin", "stop": test_date})

near_lactat = state_provider.query_data(
    "lab",
    {
        "timestamp": datetime(2025, 9, 15).date(),
        "parameter": "Lactat",
        "category": "Blutgase arteriell",
        "value_strategy": "nearest",
        "nearest_time": time(12, 0),
    },
)

# Zentral verfügbar: Zeiträume über QueryManager
time_range = state_provider.query_manager.get_time_range()
nearest_impella = state_provider.query_manager.get_nearest_impella_time()

### Data Exploration (UI)
- Views für alle Kategorien werden in der `sidebar` zugänglich gemacht
- View ruft über `state_provider.query_data()` alle verfügbaren Daten der Kategorie ab
- View bietet Auswahl für Unter-Kategorie, Parameter, Zeitraum, Value-Strategy → erneuter Aufruf von `query_data()`
- View zeigt gefilterte Daten tabellarisch und (falls numerisch) visualisiert
- Alle Views bleiben read-only und interagieren ausschließlich über den `QueryManager`

### RedCap Form (UI) - Exemplarisch für LabModel
- Voraussetzung: Datensatz + ausgewähltes Zeitfenster (`AppState.selected_time_range`, default auf Start/Ende von ECMO/Impella) + Record-ID
    - MCS-Zeiträume via `state_provider.get_device_time_ranges("impella")` (liefert `DeviceTimeRange`-Dataclasses)
- Auf Basis des Zeitfensters füllt `state_provider.data_manager` für jeden Tag `schemas.db_schemas.lab LabModel` über `query_data()`
    - `LabModel.redcap_event_name` unterscheidet baseline, impella_arm_2, ecls_arm_2
    - `LabModel.redcap_repeat_instrument` und `redcap_repeat_instance` werden automatisch gepflegt
    - Value-Strategie = "nearest" verwendet `AppState.nearest_time` (standardmäßig Implantationszeiten aus den Geräten)
- Aggregierte `LabModel`-Instanzen landen in `AppState.lab_form`

### Utilities & Wiederverwendbare Helfer
- `services/utils.py` bündelt Datums- und Zeitfunktionen (`coerce_to_datetime`, `normalize_date_range`, `expand_date_range_to_bounds`)
- Sidebar, Export Builder und DataManager greifen darauf zu, um Format-Duplikate zu vermeiden
- Hilft bei konsistenten Bounds (z.B. automatische Erweiterung auf Tagesgrenzen)

In [ ]:
from services.value_aggregation.lab_aggregator import LabAggregator
from datetime import date

state_provider.update_state(record_id="123wqewer")

# LabAggregator nutzt QueryManager-Daten und erstellt RedCap-konforme LabModel-Einträge
lab_aggregator = LabAggregator(
    state_provider, 
    date=date(2025, 9, 19), 
    record_id=state_provider.get_record_id(), 
    redcap_event_name="impella_arm_2", 
    redcap_repeat_instrument="labor", 
    redcap_repeat_instance=1, 
    value_strategy="nearest", 
    nearest_time=state_provider.get_nearest_impella_time(),
)

#### Data Aggregation / Export Builder Flow
- Über die `sidebar` öffnet der Expander "RedCap" den View `views.export_builder`
    - Zeiträume und Zeitpunkte werden hier final konfiguriert (Fallback = Geräte-Zeitbereiche aus dem QueryManager)
    - Liegt keine Record-ID vor, muss sie hier gesetzt werden (`state_provider.update_state`)
- Sobald Export-Formulare im State befüllt sind (z.B. `AppState.lab_form`), stellt der Expander passende Views bereit
- Export-Buttons greifen direkt auf `AppState.lab_form`/`…_form` zu und erzeugen CSVs

### Legacy-Cleanup
- Historische Parser-Dateien (`data_parser_backup.py`, `data_parser_base.py`) wurden entfernt; der gesamte Parser lebt in `services/data_parser.py`
- Dokumentation und README sind auf die neue Struktur (DataManager/QueryManager + Utils) ausgerichtet